In [1]:
import tensorflow as tf
import os 
import json
import pandas as pd

/Users/ds/.pyenv/versions/3.9.7/envs/ml-plg/lib/python3.9/site-packages/pandas/compat/__init__.py:109: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
dir_path='Data/Descriptions/'

In [13]:
description=[]
for fname in os.listdir(dir_path):
    description.append(json.load(open(dir_path+fname, 'r')))

In [14]:
len(description)

1000

In [15]:
description[0]

{'_id': '5436e3f3bae478396759f49a',
 '_modelType': 'image',
 'created': '2014-10-09T19:37:23.717000+00:00',
 'creator': {'_id': '5450e996bae47865794e4d0d', 'name': 'User 6VSN'},
 'dataset': {'_accessLevel': 0,
  '_id': '5a2ecc5e1165975c945942a2',
  'description': 'Moles and melanomas.\nBiopsy-confirmed melanocytic lesions. Both malignant and benign lesions are included.',
  'license': 'CC-0',
  'name': 'UDA-1',
  'updated': '2014-11-10T02:39:56.492000+00:00'},
 'meta': {'acquisition': {'image_type': 'dermoscopic',
   'pixelsX': 919,
   'pixelsY': 802},
  'clinical': {'age_approx': None,
   'anatom_site_general': 'lower extremity',
   'benign_malignant': 'benign',
   'diagnosis': 'nevus',
   'diagnosis_confirm_type': None,
   'melanocytic': True,
   'sex': None}},
 'name': 'ISIC_0000483',
 'notes': {'reviewed': {'accepted': True,
   'time': '2014-11-10T02:39:56.492000+00:00',
   'userId': '5436c6e7bae4780a676c8f93'},
  'tags': ['Challenge 2018: Task 1-2: Training',
   'Challenge 2019: T

In [16]:
df = pd.DataFrame.from_records(description)

In [17]:
df =  df.merge(pd.json_normalize(df.meta, sep="_"), left_index=True, right_index=True)

In [18]:
df = df.drop(columns=['creator', 'dataset', 'meta', 'notes'])

In [19]:
df.head(2)

,_id,_modelType,created,name,updated,acquisition_image_type,acquisition_pixelsX,acquisition_pixelsY,clinical_age_approx,clinical_anatom_site_general,clinical_benign_malignant,clinical_diagnosis,clinical_diagnosis_confirm_type,clinical_melanocytic,clinical_sex
0,5436e3f3bae478396759f49a,image,2014-10-09T19:37:23.717000+00:00,ISIC_0000483,2015-02-01T00:36:46.894000+00:00,dermoscopic,919,802,NaN,lower extremity,benign,nevus,None,True,None
1,5436e3cabae478396759f28b,image,2014-10-09T19:36:42.738000+00:00,ISIC_0000221,2015-12-04T15:06:51.949000+00:00,dermoscopic,962,671,30.0,upper extremity,benign,nevus,None,True,female


In [20]:
df.acquisition_pixelsX.max(), df.acquisition_pixelsX.min()

(4288, 576)

In [21]:
names_benign=df[df.clinical_benign_malignant=='benign'].name
names_malignant=df[df.clinical_benign_malignant=='malignant'].name

In [22]:
len(names_benign), len(names_malignant)

(841, 159)

In [24]:

for fname in names_benign:
    os.rename('Data/Images/'+fname+'.jpeg', 'Data/Images/benign/'+fname+'.jpeg')

In [25]:
for fname in names_malignant:
    os.rename('Data/Images/'+fname+'.jpeg', 'Data/Images/malignant/'+fname+'.jpeg')

In [26]:
data_set = tf.keras.preprocessing.image_dataset_from_directory(
    'Data/Images/',
    label_mode='binary',
    image_size=(df.acquisition_pixelsX.min(),df.acquisition_pixelsY.min()))

Found 1000 files belonging to 2 classes.


In [27]:
data_set

<BatchDataset shapes: ((None, 576, 540, 3), (None, 1)), types: (tf.float32, tf.float32)>

In [ ]:
def make_model(input_data):
    